In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
PROCESSED_PATH = '../data/processed/'
FEATURES_PATH = '../data/feature_engineered/'
os.makedirs(FEATURES_PATH, exist_ok=True)